Playground to test different parameters and their impact to the model and to the results

In [ ]:
import model as m
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=256)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
model= m.NN_Model()

#the multi parameters could crash, consider to lauch several times with different values
beta=[0.0]
drop=[0.4]
epochs=200
batch_size=16

#numbers to regroup the epochs (there is a parameter in tf.compile for the future case)
group_epochs=1

#unidir True if you want to standardize the yaw value to 0, update becomes True if at least 1 time the model has reach a val_loss < min_pos_error
unidir=True
update=False

results=[]
min_pos_error=3.0

for i in beta:
    for j in drop:
        #with sess.as_default():
            model.model_define_1(drop_rate=j)
            model.set_beta(i)
            model.model_compile("adam",model.pos_loss_1,['mean_absolute_error'])

            model.get_data("laser_log/datasetnowall.csv",unidir)

            print(model.data_tensor.TR_features_unidir)
            print(model.data_tensor.TS_features_unidir)
            print(model.data_tensor.TR_targets_unidir)
            print(model.data_tensor.TS_targets_unidir)

            for step in range(epochs//group_epochs):

                if step %5 == 0:
                    print("Epoch: ",step)

                history = model.model_run(group_epochs,batch_size, verbose=0, unidir=unidir)
                
                if model.num_model == 1:
                    pos_mse,pos_mae = model.get_error(verbose=1)
                    results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "position absolute error":pos_mae}) 

                if model.num_model == 2:
                    pos_mse,rot_mse,pos_mae,rot_mae = model.get_error(verbose=1)
                    results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "rotation squared error":rot_mse, "position absolute error":pos_mae, "rotation absolute error":rot_mae})  
 
                else:
                    pos_mae = min_pos_error+1

                if pos_mae < min_pos_error:
                    model.model_cnn.save_weights("models/cp-epoch-train{}{}.ckpt".format(i,j))
                    min_pos_error=pos_mae
                    update = True

print(update)

In [4]:
with open('test46k.csv', 'w') as f:
    f.write("beta;dropout;epochs;pos_mse;pos_mae;rot_mse;rot_mae\n")
    for item in results:
        f.write("{};{};{};{};{};{};{}\n".format(item["beta"],item["drop-out"],item["epochs"],item["position squared error"],item["position absolute error"],item["rotation squared error"],item["rotation absolute error"]))

#refactor the data

In [3]:
import pandas as pd

with open("data/laser_log/datasetwall.csv", "r") as data:
            laser_db = pd.read_csv(data, delimiter=";")

In [21]:
for i in range(720):
    laser_db.loc[laser_db.index%720 == i, "angle"]= np.radians((i-360)/2)

In [22]:
with open('data/datasetwallmod.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    for item in laser_db.values:
        
        f.write("{};0;{};{};{};{};{}\n".format(item[0],item[2],item[3],item[4],item[5],item[6]))


#Valutation

In [ ]:
model.model_cnn.load_weights("models/cp-epoch-train.ckpt")

In [4]:
data_test = m.DAO("laser_log/laser_log2203w.csv")

data_test.read()
data_test.divide_data(0.25)

data_tensor_test=m.DAO()

data_tensor_test.TR_features=tf.reshape(data_test.TR_features,[-1,720,1,1])
data_tensor_test.TS_features=tf.reshape(data_test.TS_features,[-1,720,1,1])
data_tensor_test.TR_targets=tf.reshape(data_test.TR_targets,[-1,3])
data_tensor_test.TS_targets=tf.reshape(data_test.TS_targets,[-1,3])

In [26]:
result = model.model_cnn.predict(data_tensor_test.TS_features)
pos_test = result[:,0:2]
rot_test = result[:,2]
pos_real = np.array([data_test.TS_targets["pos_x"].to_list(),data_test.TS_targets["pos_y"].to_list()]).T
rot_real = (data_test.TS_targets["pos_yaw"]).to_numpy()

In [24]:
pos_mae = mean_squared_error(pos_real,pos_test)
rot_mae = mean_absolute_error(rot_real,rot_test)

In [27]:
with open('predict_data2.csv', 'w') as f:
    for i in range(len(rot_real)):
        f.write("{};{};{};{};{};{}\n".format(pos_real[i][0],pos_real[i][1],rot_real[i],pos_test[i][0],pos_test[i][1],rot_test[i]))